In [ ]:
import pandas as pd
import requests
pd.set_option('display.max_columns', None)
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import datetime
import pickle
import json
import collections
import itertools

In [ ]:
df = pd.read_excel("stats2022-2023REG.xlsx")
df

In [ ]:
team_list = list(df.Team.unique())
team_list = ['BRK' if item == 'BKN' else item for item in team_list]
team_list = ['CHO' if item == 'CHA' else item for item in team_list]
team_list = ['PHO' if item == 'PHX' else item for item in team_list]
team_list

In [ ]:
number = 1
driver = webdriver.Firefox()

power_rankings = {}

while(number <= 13):
    if(number == 2):
        url = 'https://www.nba.com/news/power-rankings-week-2-2'
    elif(number == 4 or number == 5):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-la-clippers"
    elif(number == 6 or number == 8):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-golden-state-warriors-cleveland-cavaliers"
    elif(number == 7):
        url = "https://www.nba.com/news/power-rankings-week-7-golden-state-warriors-san-antonio-spurs"
    elif(number == 9):
        url = "https://www.nba.com/news/power-rankings-week-9-houston-rockets-utah-jazz"
    elif(number == 10 or number == 11):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-cleveland-cavaliers-golden-state-warriors"
    elif(number == 12):
        url = "https://www.nba.com/news/power-rankings-week-12-san-antonio-spurs-houston-rockets"
    else: 
        url = f"https://www.nba.com/news/power-rankings-week-{number}"
    driver.get(url)
    src = driver.page_source
    parser = BeautifulSoup(src, "lxml")
    ranks = parser.find_all("div", attrs = {"class": "ArticlePowerRankings_prTop__UrGiu"})
    published = parser.find('time', attrs = {"class": "ArticleHeader_ahDate__J3fwr"})
    time = published.contents[0]
    a_list = []
    for rank in ranks:
        a_tag = rank.find('a', href=True) 
        a_list.append(a_tag['href'])
    if('Updated on' in published.contents[0]):
        published.contents[0] = published.contents[0].replace('Updated on ', '')
    form = "%B %d, %Y %I:%M %p"
    new_date = datetime.datetime.strptime(published.contents[0], form).date()
    output_form = '%m-%d-%Y'
    processed_date = datetime.datetime.strftime(new_date, output_form)
    
    a_list = ['BRK' if item == '/nets' else item for item in a_list]
    a_list = ['CHO' if item == '/hornets/' else item for item in a_list]
    a_list = ['PHO' if item == '/suns/' else item for item in a_list]
    a_list = ['BOS' if item == '/celtics/' else item for item in a_list]
    a_list = ['MIL' if item == '/bucks/' else item for item in a_list]
    a_list = ['DEN' if item == '/nuggets/' else item for item in a_list]
    a_list = ['PHI' if item == '/sixers/' else item for item in a_list]
    a_list = ['CLE' if item == '/cavaliers' else item for item in a_list]
    a_list = ['MEM' if item == '/grizzlies/' else item for item in a_list]
    a_list = ['SAC' if item == '/kings' else item for item in a_list]
    a_list = ['NYK' if item == '/knicks/' else item for item in a_list]
    a_list = ['GSW' if item == '/warriors' else item for item in a_list]
    a_list = ['LAC' if item == '/clippers/' else item for item in a_list]
    a_list = ['LAL' if item == 'http://nba.com/lakers' else item for item in a_list]
    a_list = ['MIA' if item == '/heat/' else item for item in a_list]
    a_list = ['NOP' if item == '/pelicans' else item for item in a_list]
    a_list = ['OKC' if item == '/thunder' else item for item in a_list]
    a_list = ['MIN' if item == '/timberwolves' else item for item in a_list]
    a_list = ['CHI' if item == '/bulls' else item for item in a_list]
    a_list = ['TOR' if item == '/raptors/' else item for item in a_list]
    a_list = ['ATL' if item == '/hawks/' else item for item in a_list]
    a_list = ['DAL' if item == 'https://www.mavs.com' else item for item in a_list]
    a_list = ['UTA' if item == '/jazz/' else item for item in a_list]
    a_list = ['ORL' if item == '/magic' else item for item in a_list]
    a_list = ['WAS' if item == '/wizards' else item for item in a_list]
    a_list = ['IND' if item == '/pacers/' else item for item in a_list]
    a_list = ['POR' if item == '/blazers/' else item for item in a_list]
    a_list = ['HOU' if item == '/rockets' else item for item in a_list]
    a_list = ['SAS' if item == '/spurs/' else item for item in a_list]
    a_list = ['DET' if item == '/pistons' else item for item in a_list]
    
    power_rankings[processed_date] = a_list
    print('week ' + str(number) + ' done')
    
    number += 1
    
    
driver.close()
    

In [ ]:
with open("powerrankings17-18.json", "w") as f:
    json.dump(power_rankings, f)


In [ ]:
date_dict = {}

In [ ]:
driver = webdriver.Firefox()
start_date = datetime.date(2015, 10, 27)
end_date = datetime.date(2016, 4, 14)
delta = datetime.timedelta(days = 1)

#date_dict = {}

while(start_date < end_date):
    day2 = str(start_date.day)
    month2 = str(start_date.month)
    year2 = str(start_date.year)
    if(start_date.day < 10):
        day2 = "0" + day2
    if(start_date.month < 10):
        month2 = "0" + month2

    date = '' + month2 + '-' + day2 + '-' + year2  
    date_dict[date] = {}

    for team in team_list:
        url = f"https://www.basketball-reference.com/boxscores/{year2}{month2}{day2}0{team}.html"
        response = requests.get(url, headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',})
        time.sleep(3)
        print(url)
        if response.status_code == 429:
            print(int(response.headers["Retry-After"]))
            time.sleep(int(response.headers["Retry-After"]))
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            continue

        driver.get(url)
        print("hi")
        time.sleep(8)
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')

        tables = soup.find_all('table', {'id': re.compile('box-.*-game-basic')})
        advanced = soup.find_all('table', {'id': re.compile('box-.*-game-advanced')})

        away = tables[0].get('id')[4:7]

        dfs = pd.read_html(str(tables), header=1)
        advanced_dfs = pd.read_html(str(advanced), header = 1)

        df_away = dfs[0]
        df_home = dfs[1]
        df_away_advanced = advanced_dfs[0]
        df_home_advanced = advanced_dfs[1]

        df_away = pd.merge(df_away, df_away_advanced, on = ['Starters', 'MP'])
        df_home = pd.merge(df_home, df_home_advanced, on = ['Starters', 'MP'])
        
        df_away['Home'] = 0
        df_home['Home'] = 1

        date_dict[date][team] = {}
        date_dict[date][away] = {}

        date_dict[date][team][away] = df_home
        date_dict[date][away][team] = df_away

    print("day done")
    start_date += delta


driver.close()

In [ ]:
date_dict.keys()

In [ ]:
with open('dict15-16.pkl', 'wb') as f:  # open a text file
    pickle.dump(date_dict, f)
    f.close()

In [ ]:
with open('boxREG.pkl', 'wb') as f:  # open a text file
    pickle.dump(teams_dict, f)
    f.close()

In [ ]:
#seasons = ['2012-13', '2013-14', '2014-15', '2015-16','2016-17','2017-18' ,'2018-19' ,'2019-20','2021-22', '2022-23']

In [ ]:
#for season in seasons:
    #print(f"my name is {season}")

In [ ]:
#timeout = 25

In [ ]:
#driver = webdriver.Firefox()

In [ ]:
'''def get_data(seasons):
    for season in seasons:
        url = f"https://www.nba.com/stats/teams/misc?Season={season}&SeasonType=Regular+Season"
        driver.get(url)
        #element_present = EC.presence_of_element_located((By.XPATH, r"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
        #WebDriverWait(driver, timeout).until(element_present)
        #select = Select(driver.find_element("xpath", r"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
        #select.select_by_index(0)
        src = driver.page_source
        parser = BeautifulSoup(src, "lxml")
        table = parser.find("table", attrs = {"class": "Crom_table__p1iZz"})
        headers = table.findAll('th')
        headerlist = [h.text.strip() for h in headers[1:]]
        filter_header_list = [a for a in headerlist if not 'RANK' in a]
        rows = table.findAll('tr')[1:]
        player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[1:]] for i in range(len(rows))]
        #filter_header_list = filter_header_list[:-5]
        stats = pd.DataFrame(player_stats, columns = filter_header_list)
        #pd.DataFrame.to_excel(stats, f"/home/rohan/python-projects/jupyter/BasketballProject/miscteam{season}REG.xlsx")
        print(season + " done scraping")
''' 

In [ ]:
#get_data(seasons)